In [2]:
import numpy as np
from cvxpy import *
import cvxopt

In [3]:
### WITH AVERAGE DISTANCE CONSTRAINT

c = np.array([[100],[100],[100],[100]]);
n = 4 # Number of potential EV candidate Locations
x = Variable(n,boolean=True)
y = Variable((n,n),boolean=True)
w = np.array([[7],[8],[10],[9]])
#y = Variable((n, n))

d = np.array([[0,10,10,10],[0,0,10,10],[0,0,0,10],[0,0,0,0]])

def avg_dist(var_node,var_edge,dist_matrix):
    constraints = []
    tot_sum = 0
    count = 0 
    for i in range(n):
            for j in range(n):
                if i >= j:
                    constraints.append(var_edge[i,j] == 0)
                else:
                    tot_sum += dist_matrix[i,j]*var_edge[i,j]
                    count += var_edge[i,j]
                    constraints.append( ((var_node[i] + var_node[j])/2) - var_edge[i,j] == 0 )
    constraints.append(tot_sum-10*count >= 0)
    constraints.append(c.T@x >= 300)
    constraints.append(c.T@x <= 400)
    return constraints
        

prob = Problem(Minimize(c.T@x-w.T@x), avg_dist(x,y,d))
prob.solve(solver=GLPK_MI)

366.0

In [4]:
x.value

array([1., 1., 1., 1.])

In [5]:
y.value

array([[0., 1., 1., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.]])

In [84]:
avg_dist(x,y,d)

[Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, UNKNOWN, ()), Constant(CONSTANT, ZERO, ())),
 Equality(Expression(AFFINE, NONNEGATIVE, ()), Constant(CONSTANT, ZERO, ())),
 Equality(

In [6]:
### WITHOUT AVERAGE DISTANCE CONSTRAINT
def optimization_func(u, cost = 100.0):
    """ 
    This function returns the optimal EV Charging Station Locations
      
    Attributes: 
        u (float): 1d numpy array 
        cost (float): Installation cost of each charging station 
    """
    
    n = len(u) 
    c = np.ones(n)*cost
    x = Variable(n,boolean=True)

    prob = Problem(Minimize(c.T@x-w.T@x), [c.T@x >= 300,c.T@x <= 350])
    optimal_value = prob.solve(solver=GLPK_MI)
    
    return x.value, optimal_value

In [7]:
w = np.array([[7],[8],[10],[9]])
optimization_func(w)

(array([0., 1., 1., 1.]), 273.0)